In [1]:
#packages import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import pickle
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('always')  #"error", "ignore", "always", "default", "module" or "once"
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [2]:
#dataset import
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
train_data.reset_index(drop=True, inplace=True)
validation_data = pd.read_csv("validation.csv")
validation_data.reset_index(drop=True, inplace=True)

In [11]:
#import Data
X_train_X = pickle.load(open("X_train_features_X.pkl", 'rb'))
y_train_X = pickle.load(open("y_train_features_X.pkl", 'rb'))
X_validation_X = pickle.load(open("X_validation_features_X.pkl", 'rb'))
y_validation_X = pickle.load(open("y_validation_features_X.pkl", 'rb'))
X_test_X = pickle.load(open("X_test_features_X.pkl", 'rb'))

X_train = pickle.load(open("X_train_features.pkl", 'rb'))
y_train = pickle.load(open("y_train_features.pkl", 'rb'))
X_validation = pickle.load(open("X_validation_features.pkl", 'rb'))
y_validation = pickle.load(open("y_validation_features.pkl", 'rb'))
X_test = pickle.load(open("X_test_features.pkl", 'rb'))

Lin_features = pickle.load(open('bestFeatures.pkl', 'rb'))
X_features=pickle.load(open('bestFeatures_X.pkl', 'rb'))

#Liimit the input data by the feature columns
feature_X_train_L = X_train[Lin_features].values
feature_X_val_L = X_validation[Lin_features].values
feature_X_test_L = X_test[Lin_features].values

feature_X_train_X = X_train_X[X_features].values
feature_X_val_X = X_validation_X[X_features].values
feature_X_test_X = X_test_X[X_features].values

/Users/Azzurro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ResourceWarning: unclosed file <_io.BufferedReader name='X_train_features_X.pkl'>
  
/Users/Azzurro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ResourceWarning: unclosed file <_io.BufferedReader name='y_train_features_X.pkl'>
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Azzurro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='X_validation_features_X.pkl'>
  after removing the cwd from sys.path.
/Users/Azzurro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: ResourceWarning: unclosed file <_io.BufferedReader name='y_validation_features_X.pkl'>
  """
/Users/Azzurro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: ResourceWarning: unclosed file <_io.BufferedReader name='X_test_features_X.pkl'>
  
/Users/Azzurro/anaconda3/lib/python3.7/site-packages/ipyk

# CTR Estimation- Logistic

In [13]:
l_clf = LogisticRegression(C = 0.1, random_state = 123)
l_clf.fit(feature_X_train_L, y_train)

l_predict = l_clf.predict(feature_X_val_L)

l_predict_proba_list = l_clf.predict_proba(feature_X_val_L)
validation_data['click_proba_log'] = [item[1] for item in l_predict_proba_list]

l_predict_proba_list_test = l_clf.predict_proba(feature_X_test_L)
test_data['click_proba_log'] = [item[1] for item in l_predict_proba_list_test]


/Users/Azzurro/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
accuracy=accuracy_score(y_validation,l_predict)
print("Accuracy: %f" % (accuracy))

rmse = np.sqrt(mean_squared_error(y_validation, l_predict))
print("RMSE: %f" % (rmse))

roc_auc=roc_auc_score(y_validation,l_predict)
print("ROC_AUC: %f" % (roc_auc))

Accuracy: 0.995209
RMSE: 0.069215
ROC_AUC: 0.616668


# CTR Estimation- Gradient Boost  Regression Tree (GBRT)

In [15]:
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', colsample_bytree = 0.2, learning_rate = 0.1,
              max_depth = 5, alpha = 10, n_estimators = 500, random_state = 123)
xgb_clf.fit(feature_X_train_X ,y_train)

x_predict = xgb_clf.predict(feature_X_val_X )

x_predict_proba_list = xgb_clf.predict_proba(feature_X_val_X)
validation_data['click_proba_XGBOOST'] = [item[1] for item in x_predict_proba_list]

x_predict_proba_list_test = xgb_clf.predict_proba(feature_X_test_X)
test_data['click_proba_XGBOOST'] = [item[1] for item in x_predict_proba_list_test]

accuracy=accuracy_score(y_validation,x_predict)
print("Accuracy: %f" % (accuracy))

rmse = np.sqrt(mean_squared_error(y_validation, x_predict))
print("RMSE: %f" % (rmse))

roc_auc=roc_auc_score(y_validation,x_predict)
print("ROC_AUC: %f" % (roc_auc))

RMSE: 0.078879
ROC_AUC: 0.695108
Accuracy: 0.993778


# Ensembling pCTR

In [16]:
#seperate
pCTR_val_L=validation_data['click_proba_log']
pCTR_val_X=validation_data['click_proba_XGBOOST']
#avg
pCTR_avg=np.mean([pCTR_val_L,pCTR_val_X],axis=1)
#max
pCTR_max=np.max([pCTR_val_L,pCTR_val_X],axis=1)

# Non-Linear Bidding Strategy- ORTB

In [18]:
budget=6250000

In [19]:
def ORTB(c,ld,pCTR,budget,df):   
    counter = 0
    clicks = 0
    imp = 0
    bid_ORTB=np.sqrt(c*pCTR/ld + c**2)-c
    while ((budget >= 0) & (counter < len(df))):
        if pCTR[counter]>0:
            if(df['payprice'][counter] < bid_ORTB[counter]):
                budget -= df['payprice'][counter]
                clicks += df['click'][counter]
                imp += 1
        counter += 1

    return clicks, imp

In [20]:
def ORTB_2(c,ld,pCTR,budget,df):   
    counter = 0
    clicks = 0
    imp = 0
    bid_ORTB_2=c*(np.cbrt((pCTR+np.sqrt(c**2*ld**2+pCTR**2))/(c*ld))-np.cbrt(c*ld/(pCTR+np.sqrt(c**2*ld**2+pCTR**2))))
    while ((budget >= 0) & (counter < len(df))):
        if pCTR[counter]>0:
            if(df['payprice'][counter] < bid_ORTB_2[counter]):
                budget -= df['payprice'][counter]
                clicks += df['click'][counter]
                imp += 1
        counter += 1

    return clicks, imp

In [86]:
#optimum after tuning

#ORTB+Logistic= 142
c,i=ORTB(0.0103,1.18*10**-7,pCTR_val_L,budget,validation_data)
print('ORTB+Logistic: click:%d,imps:%d'%(c,i))
#ORTB+XGBOOST= 154
c,i=ORTB(0.103,5.95*10**-7,pCTR_val_X,budget,validation_data)
print('ORTB+XGBOOST: click:%d,imps:%d'%(c,i))

ORTB+Logistic: click:142,imps:153452
ORTB+XGBOOST: click:154,imps:139563


In [94]:
#parameter tuning setup
min_ld=10**-7
max_ld=10**-6
num_ld=500

min_c=0
max_c=1
num_c=500

ld_range = np.linspace(min_ld, max_ld, num_ld)
#c_range = np.linspace(min_c, max_c, num_c)
c_range=[0.01,0.04,0.07,0.1,0.13,0.16]

clicks_list = np.zeros((len(ld_range),len(c_range)))
imp_list = np.zeros((len(ld_range),len(c_range)))

In [95]:
def run_nonlinear_bid_model(df,model,pCTR):
    #implement the strategy
    for i in range(len(ld_range)):
        for j in range(len(c_range)):
            clicks,imps=model(c_range[j],ld_range[i],pCTR,budget,df)
            clicks_list[i,j]=clicks
            imp_list[i,j]=imps
            print("lambda:%d,cost:%d,max_click:%d,current_click:%d"%(i,j,clicks_list.max(),clicks_list[i,j]))
    #visualize the result
    #imps
    plt.figure(figsize = (10,8))
    plt.subplot(3,1,1)
    labels=[]
    for j in range(len(c_range)):    
        plt.plot(ld_range, imp_list[:,j],'b-')
        label='c='+str(c_range[j])
        labels.append(label)
    plt.xlabel("lambda")
    plt.ylabel("# of Impressions")
    plt.legend(labels, loc='upper left')
    
    #clicks
    plt.figure(figsize = (10,8))
    plt.subplot(3,1,1)
    labels=[]
    for j in range(len(c_range)):    
        plt.plot(ld_range, clicks_list[:,j],'r')
        label='c='+str(c_range[j])
        labels.append(label)
    plt.xlabel("lambda")
    plt.ylabel("# of Clicks")
    plt.legend(labels, loc='upper left')

    plt.show()


In [ ]:
run_nonlinear_bid_model(validation_data,ORTB,pCTR_val_X)

lambda:0,cost:0,max_click:139,current_click:139
lambda:0,cost:1,max_click:139,current_click:102
lambda:0,cost:2,max_click:139,current_click:85
lambda:0,cost:3,max_click:139,current_click:75
lambda:0,cost:4,max_click:139,current_click:68
lambda:0,cost:5,max_click:139,current_click:64
lambda:1,cost:0,max_click:139,current_click:138
lambda:1,cost:1,max_click:139,current_click:103
lambda:1,cost:2,max_click:139,current_click:86
lambda:1,cost:3,max_click:139,current_click:76
lambda:1,cost:4,max_click:139,current_click:69
lambda:1,cost:5,max_click:139,current_click:64
lambda:2,cost:0,max_click:139,current_click:138
lambda:2,cost:1,max_click:139,current_click:106
lambda:2,cost:2,max_click:139,current_click:87
lambda:2,cost:3,max_click:139,current_click:77
lambda:2,cost:4,max_click:139,current_click:69
lambda:2,cost:5,max_click:139,current_click:65
lambda:3,cost:0,max_click:139,current_click:137
lambda:3,cost:1,max_click:139,current_click:106
lambda:3,cost:2,max_click:139,current_click:86
lambd

In [64]:
run_nonlinear_bid_model(validation_data,ORTB_2,pCTR_val_X)

In [ ]:
run_nonlinear_bid_model(validation_data,ORTB,pCTR_val_L)

In [ ]:
run_nonlinear_bid_model(validation_data,ORTB_2,pCTR_val_X)

# Leaderboard Bid

In [78]:
pCTR_test_X=test_data['click_proba_XGBOOST']

In [79]:
#Will output the csv which will be used for the leaderboard website
c_optimum=0.103
ld_optimum=5.95*10**-7
test_data['bidprice'] = round(np.sqrt(c_optimum*pCTR_test_X/ld_optimum + c_optimum**2)-c_optimum)

leaderboard_output = test_data[['bidid','bidprice']]

leaderboard_output = leaderboard_output.sort_values(by='bidprice', ascending=True)
leaderboard_output.to_csv("testing bidding price_v1.csv", index=False)

leaderboard_output.head(10)

,bidid,bidprice
15964,0bff28fdd290fea4bfa3bd7a6981e72a260af732,1.0
170007,486da4677f0a11275369f95abc81c2fe7dd0db49,1.0
53369,6e397c59c551bb9b3539fd9e5c6e6c894ba5209e,1.0
236585,2be3f65d4c47b33c39d64bd21feea3eb1b54f614,1.0
40569,089719ddaf842b89bcef2133568dc32a29e64d68,1.0
209830,931583b1a71cb59fc55319bf5429c8f01f73b09f,1.0
23981,210540ff360b33ec6e75dc28684bf38e96c0a9c7,1.0
201966,5e397e0f127f7fa8de3eee432a0ae4f9ca92534a,1.0
290992,9e7e8f322f17d62b49bccde983f3d3f521523399,1.0
193498,015feb0b514e9012d4628a0ddc968fb7059a6ce0,1.0
